In [3]:
import pandas as pd
import numpy as np
import torch

def read_file(fname: str) -> pd.DataFrame:
    """Reads a filename and formats it properly for simpletransformers"""
    df = pd.read_table(fname, sep="\t", header=None, names="text,labels,role".split(","))
    offensive_ids = df.labels != "Acceptable speech"

    df.labels[offensive_ids] = 1
    df.labels[~offensive_ids] = 0
    
    df["labels"] = df.labels.astype(np.int8)
    df = df.drop(columns=["role"])
    return df

def fine_tune_and_evaluate(
    model_type,
    model_name,
    language
                            ):
    import torch
    torch.cuda.empty_cache()
    if lang not in {"sl", "hr", "en"}:
        raise AttributeError(f"Language {lang} is not valid")
    eval_file, train_file = f"../data/lgbt-{lang}.test.tsv" , f"../data/lgbt-{lang}.train.tsv"
    train = read_file(train_file)
    test = read_file(eval_file)
    
    from simpletransformers.classification import ClassificationModel
    model_args = {
        "num_train_epochs": 5,
        "learning_rate": 1e-5,
        "overwrite_output_dir": True,
        "train_batch_size": 40,
        "no_save": True,
    }

    model = ClassificationModel(
        model_type, model_name, use_cuda=True,
        args=model_args

    )
    model.no_save  = True
    model.overwrite_output_dir = True
    model.train_model(train, )

    #print(model.eval_model(test))
    from sklearn.metrics import accuracy_score, f1_score
    y_true = test["labels"]
    y_pred = model.predict(list(test["text"].values))[0]

    accuracy = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred, average="macro")
    return accuracy, f1


In [4]:
config_dict = {
    "sl": [
        ("IMSyPP/hate_speech_slo", "bert"),
        ("xlm-roberta-base", "xlmroberta"),
        ("EMBEDDIA/sloberta", "camembert"),
        ("EMBEDDIA/sloberta", "roberta"),
        ("EMBEDDIA/crosloengual-bert", "bert"),
        ],
    "hr": [
        ("xlm-roberta-base", "xlmroberta"),
        ("classla/bcms-bertic", "electra"),
        ("EMBEDDIA/crosloengual-bert", "bert"),
    ],
    "en": [
        ("xlm-roberta-base", "xlmroberta"),
        ("xlm-roberta-large", "xlmroberta"),
        ("roberta-base", "roberta"),
    ]
}
output = ""
output += "|model name| model type | language | accuracy | macro F1|\n"
output += "|---|---|---|---|---|\n"
for lang, confs in config_dict.items():
    for conf in confs:
        print(output)
        model_name, model_type = conf
        try:
            a, f = fine_tune_and_evaluate(model_type, model_name, lang)
            output += f"|{model_name}|{model_type}|{lang}|{a:0.3f}|{f:0.3f}|\n"
        except Exception as e:
            #raise e
            output += f"|{model_name}|{model_type}|{lang}|Error|{e}|\n"
print(output)

|model name| model type | language | accuracy | macro F1|
|---|---|---|---|---|



/home/peterr/anaconda3/lib/python3.8/site-packages/simpletransformers/classification/classification_model.py:941: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  torch.nn.utils.clip_grad_norm_(
/home/peterr/anaconda3/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:129: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optim


|model name| model type | language | accuracy | macro F1|
|---|---|---|---|---|
|IMSyPP/hate_speech_slo|bert|sl|0.578|0.578|



Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.weig


|model name| model type | language | accuracy | macro F1|
|---|---|---|---|---|
|IMSyPP/hate_speech_slo|bert|sl|0.578|0.578|
|xlm-roberta-base|xlmroberta|sl|0.623|0.622|



...


|model name| model type | language | accuracy | macro F1|
|---|---|---|---|---|
|IMSyPP/hate_speech_slo|bert|sl|0.578|0.578|
|xlm-roberta-base|xlmroberta|sl|0.623|0.622|
|EMBEDDIA/sloberta|camembert|sl|0.717|0.717|
|EMBEDDIA/sloberta|roberta|sl|Error|Can't load tokenizer for 'EMBEDDIA/sloberta'. Make sure that:

- 'EMBEDDIA/sloberta' is a correct model identifier listed on 'https://huggingface.co/models'

- or 'EMBEDDIA/sloberta' is the correct path to a directory containing relevant tokenizer files

|
|EMBEDDIA/crosloengual-bert|bert|sl|0.692|0.692|
|xlm-roberta-base|xlmroberta|hr|0.759|0.745|
|classla/bcms-bertic|electra|hr|0.859|0.844|
|EMBEDDIA/crosloengual-bert|bert|hr|0.846|0.831|
|xlm-roberta-base|xlmroberta|en|0.816|0.746|



Some weights of the model checkpoint at xlm-roberta-large were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.we


|model name| model type | language | accuracy | macro F1|
|---|---|---|---|---|
|IMSyPP/hate_speech_slo|bert|sl|0.578|0.578|
|xlm-roberta-base|xlmroberta|sl|0.623|0.622|
|EMBEDDIA/sloberta|camembert|sl|0.717|0.717|
|EMBEDDIA/sloberta|roberta|sl|Error|Can't load tokenizer for 'EMBEDDIA/sloberta'. Make sure that:

- 'EMBEDDIA/sloberta' is a correct model identifier listed on 'https://huggingface.co/models'

- or 'EMBEDDIA/sloberta' is the correct path to a directory containing relevant tokenizer files

|
|EMBEDDIA/crosloengual-bert|bert|sl|0.692|0.692|
|xlm-roberta-base|xlmroberta|hr|0.759|0.745|
|classla/bcms-bertic|electra|hr|0.859|0.844|
|EMBEDDIA/crosloengual-bert|bert|hr|0.846|0.831|
|xlm-roberta-base|xlmroberta|en|0.816|0.746|
|xlm-roberta-large|xlmroberta|en|0.845|0.787|



Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.out


|model name| model type | language | accuracy | macro F1|
|---|---|---|---|---|
|IMSyPP/hate_speech_slo|bert|sl|0.578|0.578|
|xlm-roberta-base|xlmroberta|sl|0.623|0.622|
|EMBEDDIA/sloberta|camembert|sl|0.717|0.717|
|EMBEDDIA/sloberta|roberta|sl|Error|Can't load tokenizer for 'EMBEDDIA/sloberta'. Make sure that:

- 'EMBEDDIA/sloberta' is a correct model identifier listed on 'https://huggingface.co/models'

- or 'EMBEDDIA/sloberta' is the correct path to a directory containing relevant tokenizer files

|
|EMBEDDIA/crosloengual-bert|bert|sl|0.692|0.692|
|xlm-roberta-base|xlmroberta|hr|0.759|0.745|
|classla/bcms-bertic|electra|hr|0.859|0.844|
|EMBEDDIA/crosloengual-bert|bert|hr|0.846|0.831|
|xlm-roberta-base|xlmroberta|en|0.816|0.746|
|xlm-roberta-large|xlmroberta|en|0.845|0.787|
|roberta-base|roberta|en|0.839|0.784|



In [1]:
# This demonstrates that it is possible to use xlm models as well, 
#but using a different spelling (not "xlm-roberta" as stated in 
#the config.json but rather as "xmlroberta").

from simpletransformers.classification import ClassificationModel
model_args = {
    "num_train_epochs": 5,
    "learning_rate": 1e-5,
    "overwrite_output_dir": True,
    "train_batch_size": 40,
    "no_save": True,
}

model = ClassificationModel(
    "xlmroberta", "xlm-roberta-base", use_cuda=True,
    args=model_args

)

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.weig

/home/peterr/anaconda3/lib/python3.8/site-packages/simpletransformers/classification/classification_model.py:489: UserWarning: use_multiprocessing automatically disabled as xlmroberta fails when using multiprocessing for feature conversion.
  warnings.warn(
